In [1]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Markdown

In [2]:
## when start the server: OLLAMA_CONTEXT_LENGTH=8192 ollama serve

OLLAMA_BASE_URL = "http://localhost:8888"  # URL par défaut d'Ollama
# OLLAMA_BASE_URL = "http://localhost:11434"  # URL par défaut d'Ollama

# OLLAMA_MODEL = "llama3.3:latest"
# OLLAMA_MODEL = "deepseek-r1:14b"
# OLLAMA_MODEL = "phi4:latest"
OLLAMA_MODEL = "mistral"


llm = ChatOllama(model=OLLAMA_MODEL, temperature=0.0, base_url=OLLAMA_BASE_URL)

In [3]:
question = "How to write a langGraph project which simulates a developer team composed by an analyst, a coder, a tester and a reviewer?"

# Simple usage of ollama model

In [4]:
analyst = llm.invoke(question)

In [5]:
display(Markdown(analyst.content))

 To create a langGraph project that simulates a software development team consisting of an Analyst, Coder, Tester, and Reviewer, you can follow these steps:

1. Define the roles and responsibilities for each member in your graph:

- Analyst (Node A): Gathers requirements, creates user stories, and defines tasks.
- Coder (Node C): Writes code based on the tasks defined by the analyst.
- Tester (Node T): Tests the code written by the coder to ensure it meets the requirements.
- Reviewer (Node R): Reviews the code for quality and adherence to best practices.

2. Create edges between nodes to represent the workflow:

- Analyst -> Coder: The analyst creates tasks that are assigned to the coder.
- Coder -> Tester: Once the coder completes a task, they pass it on to the tester for testing.
- Tester -> Reviewer: If the test results are positive, the tester passes the completed task to the reviewer for code review.
- Reviewer -> Analyst: The reviewer returns the code to the analyst if there are any issues or suggestions for improvement. This creates a loop in the graph until the code is approved and ready for deployment.

3. Implement the simulation using langGraph:

- Initialize your project with the necessary nodes, edges, and properties (e.g., task status, priority, etc.).
- Write functions to simulate the actions of each role (e.g., createTask(), writeCode(), testCode(), reviewCode()).
- Implement a loop or scheduler to run the simulation over time, allowing each role to perform their tasks based on the current state of the graph.
- Add randomness or variability to the simulation to make it more realistic (e.g., varying task completion times, code quality issues, etc.).

4. Visualize the simulation using langGraph's built-in visualization tools or integrate with external libraries for a more interactive experience.

5. Analyze the results of your simulation to gain insights into team performance, bottlenecks, and areas for improvement. You can use various metrics such as task completion time, code quality, and collaboration efficiency to evaluate the effectiveness of your simulated development team.

## More complex prompt

In [6]:
PROMPT = """You are a professional analyst. You read the question and split it into several steps to be able to analyse it in depth and provide the more precise answer you can.

You have to follow meticulously the following structure:

- step 1: ...
- step 2: ...
...
- step n: ...

Your answer will be writing in markdown format.
You will suggest Python implementation when it's possible.

"""

In [7]:
analyst_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", PROMPT),
        ("human", "The question to analyse is: {question}"),
    ]
)

In [8]:
analyst = llm.invoke(analyst_prompt.invoke({"question": question}))

In [9]:
display(Markdown(analyst.content))

1. Understand the Requirements:
   - The project aims to simulate a developer team consisting of four roles: Analyst, Coder, Tester, and Reviewer.
   - Each role will have specific tasks associated with them in the software development lifecycle (SDLC).

2. Define Roles and Tasks:
   - Analyst: Gather requirements, create design documents, and define user stories.
     - Python Implementation: Create a class `Analyst` with methods for gathering requirements, creating design docs, and defining user stories.
   - Coder: Write code based on the provided design documents and user stories.
     - Python Implementation: Create a class `Coder` with methods for writing code based on given design docs and user stories.
   - Tester: Test the code to ensure it meets the requirements and is free of bugs.
     - Python Implementation: Create a class `Tester` with methods for testing the code, reporting bugs, and verifying fixes.
   - Reviewer: Review the code for quality, adherence to best practices, and compliance with design documents.
     - Python Implementation: Create a class `Reviewer` with methods for reviewing the code, providing feedback, and approving or rejecting changes.

3. Define Workflow:
   - The workflow should be iterative, allowing each role to complete their tasks in sequence before moving on to the next one.
     - Python Implementation: Implement a loop that allows each role to perform their tasks in order (Analyst -> Coder -> Tester -> Reviewer) and then repeat the cycle for multiple iterations.

4. Implement Communication Mechanisms:
   - Each role should be able to communicate with others, reporting progress, issues, and feedback.
     - Python Implementation: Implement communication mechanisms such as functions or methods that allow roles to send messages to each other.

5. Add Randomness for Realism:
   - To make the simulation more realistic, add randomness in task completion times, bug rates, and reviewer feedback.
     - Python Implementation: Use libraries like `random` to introduce randomness in the simulation.

6. Evaluate Performance Metrics:
   - Measure the performance of each role and the team as a whole by tracking metrics such as time-to-completion, bug rates, and code quality.
     - Python Implementation: Create functions to calculate these metrics and log them for analysis.

7. Visualize Results:
   - Present the results in a clear and understandable manner using data visualization techniques.
     - Python Implementation: Use libraries like `matplotlib` or `seaborn` to create graphs and charts that illustrate the performance metrics.

8. Iterate and Improve:
   - Continuously iterate on the simulation, making improvements based on the results and feedback received.
     - Python Implementation: Regularly update the code to address any issues or inefficiencies identified during the simulation.

## Add the RAG

In [43]:
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup as Soup
import markdownify

urls = [
    "https://langchain-ai.github.io/langgraph/tutorials",
    "https://langchain-ai.github.io/langgraph/concepts",
    "https://langchain-ai.github.io/langgraph/how-tos",
]

def bs4_extractor(html: str) -> str:
    soup = Soup(html, "html.parser")
    for script in soup(["script", "style", "nav"]):
        script.decompose()
    return markdownify.markdownify(str(soup), heading_style="ATX")

docs = [RecursiveUrlLoader(url, extractor=bs4_extractor, max_depth=3).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]


In [44]:
print(len(docs_list))
for d in docs_list:
    print(d.metadata['source'])

147
https://langchain-ai.github.io/langgraph/tutorials
https://langchain-ai.github.io/langgraph/tutorials/
https://langchain-ai.github.io/langgraph/tutorials/chatbots/information-gather-prompting/
https://langchain-ai.github.io/langgraph/tutorials/tnt-llm/tnt-llm/
https://langchain-ai.github.io/langgraph/tutorials/code_assistant/langgraph_code_assistant/
https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/
https://langchain-ai.github.io/langgraph/tutorials/deployment/
https://langchain-ai.github.io/langgraph/tutorials/rewoo/rewoo/
https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag_local/
https://langchain-ai.github.io/langgraph/tutorials/auth/getting_started/
https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag_local/
https://langchain-ai.github.io/langgraph/tutorials/workflows/
https://langchain-ai.github.io/langgraph/tutorials/llm-compiler/LLMCompiler/
https://langchain-ai.github.io/langgraph/tutorials/introduction/
ht

In [5]:

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)


In [6]:
len(doc_splits)

28498

### Store the data in the database

In [10]:
import chromadb
client = chromadb.PersistentClient(path="./chroma-rag")

In [11]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

# EMBED_MODEL= "nomic-embed-text:latest"
EMBED_MODEL= "mxbai-embed-large"
llm_embed = OllamaEmbeddings(model=EMBED_MODEL, base_url=OLLAMA_BASE_URL)

# Add to vectorDB
vectorstore = Chroma(
    collection_name="langgraph-rag",
    embedding_function=llm_embed,
    persist_directory="./chroma-rag"
)


In [9]:
vectorstore.add_documents(doc_splits)

['a0944c5e-87b6-40d8-9c4f-b27d1f7be17c',
 '48fe2c06-39a1-4b90-97f9-db947a9e33cb',
 'dcd16d6f-dbf6-4d22-b1ad-1be4047cf0f4',
 '0f6875ed-f742-4cdc-be1b-ace57d65d4a2',
 '8a2c0f4d-da9a-4471-a757-2169c37bac83',
 '66831f77-306a-4888-82a2-79d2b99e2aff',
 '0c569a12-f10a-4b07-a134-f4ad57956437',
 '03b5fc01-f96f-499b-a873-10871969706f',
 '0a4073b7-9061-4bc3-b7b8-177299fadd6c',
 '7cdb6179-c913-4bf3-8683-2e8d6d57d4df',
 '89f1b1e8-fcbc-44fb-9a01-a7d426c45a66',
 '9a7a3f03-7215-44e0-a900-0b36e22af41a',
 'c9ff1855-7900-4717-8cf5-f62110a4c4de',
 'e866dbe1-1991-414e-9aa0-92b41cf6a1db',
 '406f575c-53ea-448e-89f5-d75752e3e8fd',
 '8882f2bf-91bf-4a44-807b-19c9e689c608',
 'c9c461a3-df50-4e7f-a908-01030cd1b385',
 '27bbc098-6805-4367-9683-96f38964d514',
 'd3446378-5990-4069-abbc-627d2f2d3624',
 '678c191d-37dc-4b0d-a9d4-f7658f549384',
 '1cce459a-79da-450e-9aab-9a3ec2ba0ba4',
 '0c45dc92-cdc3-4b32-85c2-ad11ac372141',
 '02fa38f0-0ad1-4e0d-9dd9-e64e3c9be2a2',
 'cfdabf59-fa20-44e0-b5d1-9b9bdd20031b',
 '79242a09-9fe0-

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
context = retriever.invoke(question)

In [14]:
PROMPT = """You are a professional analyst. You read the question and split it into several steps to be able to analyse it in depth and provide the more precise answer you can.

You can use the following context to help you answer the question:
{context}

You have to follow meticulously the following structure:

- step 1: ...
- step 2: ...
...
- step n: ...

Your answer will be writing in markdown format.
You will suggest Python implementation when it's possible.

"""

In [15]:
analyst_with_rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", PROMPT),
        ("human", "The question to analyse is: {question}"),
    ]
)

In [24]:
OLLAMA_MODEL = "mistral"
llm = ChatOllama(model=OLLAMA_MODEL, temperature=0.0, base_url=OLLAMA_BASE_URL)

In [27]:
analyst = llm.invoke(analyst_with_rag_prompt.invoke({"question": question, "context": "\n".join([c.page_content for c in context])}))

In [28]:
display(Markdown(analyst.content))

1. **Define the agents' tools**: Each team member will have access to different tools to perform their tasks effectively. Here are the tools for each team member:

   - Analyst: Search engine (e.g., Google), Data analysis tools (e.g., Excel, Python libraries like pandas or NumPy)
   - Coder: Code editor (e.g., Visual Studio Code), Version control system (e.g., Git), Package manager (e.g., npm for Node.js projects, pip for Python projects)
   - Tester: Testing frameworks (e.g., Jest for JavaScript, PyTest for Python), Debugging tools (e.g., Chrome DevTools, GDB)
   - Reviewer: Code review tools (e.g., GitHub's pull request reviews, SonarQube)

   You can create these tools as functions or classes in your project, depending on the complexity and requirements.

2. **Define some utilities to help create the graph and agents**: To manage the flow of tasks within the team, you will need utility functions for creating the state graph, adding nodes (representing tasks), adding edges (representing dependencies between tasks), and defining conditions for task execution.

3. **Create and define each team**: Now that you have defined the tools and utilities, you can create and define each team. Here's an example of how to create the DeveloperTeam:

   ```python
   from typing import Dict, List
   from langchain_core.messages import HumanMessage
   from langgraph.prebuilt import create_react_agent

   # Define the tools for each team member
   analyst_tools = [search_engine_tool, data_analysis_tool]
   coder_tools = [code_editor_tool, version_control_system_tool, package_manager_tool]
   tester_tools = [testing_framework_tool, debugging_tool]
   reviewer_tools = [code_review_tool]

   # Define the team members as agents
   analyst_agent = create_react_agent(llm, tools=analyst_tools)
   coder_agent = create_react_agent(llm, tools=coder_tools)
   tester_agent = create_react_agent(llm, tools=tester_tools)
   reviewer_agent = create_react_agent(llm, tools=reviewer_tools)

   # Define the team supervisor
   developer_team_supervisor = SupervisorNode(llm, [analyst_agent, coder_agent, tester_agent, reviewer_agent])
   ```

4. **Compose everything together**: Finally, you can compose the entire project by creating the state graph and adding nodes (tasks) and edges (dependencies) between them. You can also define conditions for task execution based on the team's requirements. Here's an example of how to create the DeveloperTeam project:

   ```python
   from langgraph.graph import StateGraph, START, END

   graph = StateGraph(DeveloperTeamState)
   graph.add_node("analyze", analyze)
   graph.add_node("code", code)
   graph.add_node("test", test)
   graph.add_node("review", review)

   graph.add_edge("analyze", "code")
   graph.add_edge("code", "test")
   graph.add_edge("test", "review")

   def should_review(state: DeveloperTeamState) -> str:
       # Define conditions for review task execution based on the team's requirements
       if state["code_quality"] >= 90:
           return "review"
       return END

   graph.add_conditional_edges(
       "review",
       should_review,
       {"review": "review", END: END}
   )
   graph.add_edge(START, "analyze")
   app = graph.compile()
   ```

## And with DeepSeek ?

In [19]:
OLLAMA_MODEL = "deepseek-r1:7b"
llm = ChatOllama(model=OLLAMA_MODEL, temperature=0.0, base_url=OLLAMA_BASE_URL)

In [22]:
analyst = llm.invoke(analyst_with_rag_prompt.invoke({"question": question, "context": "\n".join([c.page_content for c in context])}))

In [23]:
display(Markdown(analyst.content))

<think>
Okay, so I need to figure out how to create a LangGraph project that simulates a developer team consisting of an analyst, a coder, a tester, and a reviewer. Hmm, let me break this down step by step.

First, I remember from the context provided that LangGraph is used for creating multi-agent applications with conversational agents. Each agent can have tools and perform actions based on certain conditions. So, each role in our team (analyst, coder, tester, reviewer) will be an agent with specific capabilities.

Let me start by outlining what each role might need:

1. **Analyst**: They should be able to gather information from various sources like the web or APIs. Maybe they can perform web scraping or data extraction.
2. **Coder**: They'll need to write code, perhaps using Python. They could use tools for code generation or debugging.
3. **Tester**: Their role is to test the application's functionality. They might run automated tests and report issues.
4. **Reviewer**: They should review the work, maybe format documents or ensure code quality.

Now, thinking about LangGraph's structure, each agent will be a node in the state graph. Each node can have multiple incoming edges from other nodes, representing how tasks are passed between agents based on certain conditions.

I need to define tools for each agent. For example, an analyst might use web scraping tools or APIs to gather data. A coder could use code generators like LangChain's HuggingFaceEmbeddings or ArXivExtractor. The tester might run LLM validations using Anthropic's API. The reviewer could format documents with Marked or Prettify.

Next, I'll need to create the agents themselves. Each agent will have their own set of tools and a supervisor node that coordinates them. For instance, the analyst's supervisor can decide when to pass information to the coder based on certain conditions like data availability.

I should also consider how these agents interact. Maybe the coder waits for data from the analyst before generating code. The tester could run validations after coding is done. The reviewer might format the output or ensure it meets quality standards.

Putting this together, I'll structure each agent with their tools and supervisor nodes. Then, connect them in a state graph where each node's actions depend on conditions like data availability or task completion status.

Wait, but how do I handle the flow between agents? Maybe using conditional edges based on whether an action has been completed successfully. For example, if the analyst gathers enough data (condition met), then the coder can proceed to write code.

I also need to ensure that each agent's supervisor knows when their task is done and passes the necessary information along. This might involve updating messages or statuses within the LangGraph framework.

Another thing to consider is error handling. Each agent should be able to handle failures gracefully, maybe retrying tasks or informing the team about issues.

Finally, I'll need to compile the state graph into an application that can run these agents in a coordinated manner. Testing each part as I go would help catch any issues early on.
</think>

To create a LangGraph project simulating a developer team with roles such as analyst, coder, tester, and reviewer, follow this structured approach:

### Step-by-Step Explanation

1. **Define Roles and Tools**
   - Each role (analyst, coder, tester, reviewer) is represented by an agent with specific tools.
     - **Analyst**: Uses web scraping or APIs to gather data.
     - **Coder**: Employs code generation tools like LangChain's HuggingFaceEmbeddings.
     - **Tester**: Implements automated testing using LLM validations (e.g., Anthropic API).
     - **Reviewer**: Focuses on document formatting and quality assurance.

2. **Create Agents**
   - Each agent is defined with their tools and supervisor nodes to coordinate tasks based on conditions.
     ```python
     from langchain_core.messages import HumanMessage
     from langchain_openai import ChatOpenAI

     # Analyst Agent
     analyst_agent = create_react_agent(llm, tools=[analyst_tool])
     
     # Coder Agent
     coder_agent = create_react_agent(llm, tools=[coder_tool])
     
     # Tester Agent
     tester_agent = create_react_agent(llm, tools=[tester_tool])
     
     # Reviewer Agent
     reviewer_agent = create_react_agent(llm, tools=[reviewer_tool])
     ```

3. **Build the State Graph**
   - Structure agents as nodes in a state graph with conditional edges.
     ```python
     from langgraph.graph import StateGraph, START, END

     graph = StateGraph(TaxonomyGenerationState)
     
     # Define nodes and their actions
     graph.add_node("gather_data", gather_data)
     graph.add_node("write_code", write_code)
     graph.add_node("run_tests", run_tests)
     graph.add_node("format_review", format_review)

     # Establish flow based on conditions
     graph.add_edge("gather_data", "write_code", condition=lambda state: len(state["data"]) > 0)
     graph.add_edge("write_code", "run_tests")
     graph.add_edge("run_tests", "format_review")
     graph.add_edge("format_review", END)

     # Ensure review depends on testing
     graph.add_conditional_edges(
         "format_review",
         lambda state: len(state["test_results"]) > 0,
         {"format_review": "format_review", "review": "review"}
     )

     graph.add_edge(START, "gather_data")
     ```

4. **Implement Conditional Logic**
   - Use conditions to control task execution flow.
     ```python
     def should_review(state: TaxonomyGenerationState) -> str:
         if state["test_results"]:
             return "format_review"
         else:
             return "review"

     graph.add_conditional_edges(
         "write_code",
         lambda state: len(state["code"]) > 0,
         {"write_code": "write_code", "review": "review"}
     )
     ```

5. **Coordinate Agents with Supervisors**
   - Each agent's supervisor node ensures tasks are initiated based on conditions.
     ```python
     from langgraph.utils import get_message

     def supervisor_analyze(state: TaxonomyGenerationState) -> str:
         if len(state["data"]) < 10:
             return "gather_data"
         else:
             return "write_code"

     graph.add_edge(START, "gather_data", condition=supervisor_analyze)
     ```

6. **Compile and Run the Application**
   - Execute the compiled state graph to simulate team interactions.
     ```python
     app = graph.compile()
     result = app.run()
     print(result)
     ```

### Summary

By structuring each role as an agent with specific tools and coordinating their execution through a state graph, you can simulate a developer team's workflow. Each step involves defining agents, creating conditional edges based on task dependencies, and compiling the graph for execution. This approach allows for dynamic task assignment and coordination among team members, ensuring efficient problem-solving and workflow management.